In [5]:
import os  
import json 
import pandas as pd 
import traceback

In [2]:
!pip install langchain_community

In [6]:
from langchain_openai import ChatOpenAI

In [7]:
from dotenv import load_dotenv
load_dotenv()
key=os.getenv('OPENAI_API_KEY')

In [8]:
model =ChatOpenAI(openai_api_key=key, model_name="gpt-3.5-turbo", temperature=0.5)

In [7]:
!pip install -U langchain-openai

   ---------------------------------------- 0.0/798.9 kB ? eta -:--:--
   --------------------------------------- 798.9/798.9 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.25
    Uninstalling langchain-core-0.2.25:
      Successfully uninstalled langchain-core-0.2.25


In [9]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}
template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""
quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=template
)

In [11]:
from langchain_core.output_parsers.json import SimpleJsonOutputParser
# quiz_chain=quiz_generation_prompt | model | SimpleJsonOutputParser
quiz_chain=LLMChain(llm=model, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

c:\Users\LENOVO\Desktop\projects\mcq_generator\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"], 
    template=TEMPLATE2
)

In [14]:
review_chain=LLMChain(llm=model, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)
# review_chain=quiz_evaluation_prompt | model | SimpleJsonOutputParser

In [15]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], 
                                        input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], 
                                        verbose=True
                                    )

In [2]:
with open("../data.txt", 'r') as file:
    text=file.read()

In [3]:
text

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [16]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [17]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": text,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

c:\Users\LENOVO\Desktop\projects\mcq_generator\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [18]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1372
Prompt Tokens:1004
Completion Tokens:368
Total Cost:0.001054


In [19]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [23]:
quiz=response.get("quiz")
quiz

'\n{\n"1": {"mcq": "What is the scientific study of life called?", "options": {"a": "Chemistry", "b": "Biology", "c": "Physics", "d": "Geology"}, "correct": "b"},\n"2": {"mcq": "Which theme in biology explains the unity and diversity of life?", "options": {"a": "Evolution", "b": "Photosynthesis", "c": "Cellular respiration", "d": "Mitosis"}, "correct": "a"},\n"3": {"mcq": "What is the basic unit of life that all organisms are made up of?", "options": {"a": "Atoms", "b": "Molecules", "c": "Cells", "d": "Tissues"}, "correct": "c"},\n"4": {"mcq": "Which process is important for organisms to move, grow, and reproduce?", "options": {"a": "Photosynthesis", "b": "Evolution", "c": "Energy processing", "d": "Cell division"}, "correct": "c"},\n"5": {"mcq": "What is the age of life on Earth according to the text?", "options": {"a": "2.5 billion years", "b": "3.7 billion years", "c": "5 billion years", "d": "1 billion years"}, "correct": "b"}\n}'

In [24]:
quiz=json.loads(quiz)

In [25]:
quiz_table_data=[]
for key, val in quiz.items():
    mcq=val["mcq"]
    options=" | ".join(
        [f"{option}:{option_value}" for option, option_value in val["options"].items()]
    )
    correct=val["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})

In [26]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life called?',
  'Choices': 'a:Chemistry | b:Biology | c:Physics | d:Geology',
  'Correct': 'b'},
 {'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'Choices': 'a:Evolution | b:Photosynthesis | c:Cellular respiration | d:Mitosis',
  'Correct': 'a'},
 {'MCQ': 'What is the basic unit of life that all organisms are made up of?',
  'Choices': 'a:Atoms | b:Molecules | c:Cells | d:Tissues',
  'Correct': 'c'},
 {'MCQ': 'Which process is important for organisms to move, grow, and reproduce?',
  'Choices': 'a:Photosynthesis | b:Evolution | c:Energy processing | d:Cell division',
  'Correct': 'c'},
 {'MCQ': 'What is the age of life on Earth according to the text?',
  'Choices': 'a:2.5 billion years | b:3.7 billion years | c:5 billion years | d:1 billion years',
  'Correct': 'b'}]

In [27]:
quiz=pd.DataFrame(quiz_table_data)

In [28]:
quiz.to_csv("ml.csv", index=False)

In [29]:
from datetime import datetime

LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [30]:
LOG_FILE

'08_05_2024_20_01_41.log'